In [6]:
import pandas as pd
import os 
from pathlib import Path
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error 
pd.set_option('display.max_columns', 200)

In [2]:
os.getcwd()

'/Users/macintosh/github/data-science-project'

In [3]:
INPUT_DIR = "/Users/Macintosh/Desktop/dataset/predict-energy-behavior-of-prosumers/"
# INPUT_DIR = Path("/Users/Macintosh/Downloads/predict-energy-behavior-of-prosumers/")
TEST_INPUT_DIR = "/Users/Macintosh/Desktop/dataset/predict-energy-behavior-of-prosumers/example_test_files/"


In [4]:
# Train Data
train_df = pd.read_csv(INPUT_DIR + "train.csv", sep= ",")
# train_df = pd.read_csv(INPUT_DIR / "train.csv", sep= ",")
client_df = pd.read_csv(INPUT_DIR + "client.csv", sep= ",")
electricity_prices_df = pd.read_csv(INPUT_DIR + "electricity_prices.csv", sep= ",")
forecast_weather_df = pd.read_csv(INPUT_DIR + "forecast_weather.csv", sep= ",")
gas_prices_df = pd.read_csv(INPUT_DIR + "gas_prices.csv", sep= ",")
historical_weather_df = pd.read_csv(INPUT_DIR + "historical_weather.csv", sep= ",")
weather_station_to_county_mapping_df = pd.read_csv(INPUT_DIR + "weather_station_to_county_mapping.csv", sep= ",")

In [5]:
# Test Data
test_df = pd.read_csv(TEST_INPUT_DIR + "test.csv")
sample_submission_df = pd.read_csv(TEST_INPUT_DIR + "sample_submission.csv")

In [265]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression

In [499]:
# Create data frame for predict value
# ALL_IDS = clean_df.row_id.unique()
# oof = pd.DataFrame(data=np.zeros((len(ALL_IDS), 1)), index=ALL_IDS, columns=['target'])
oof = train_df.loc[:, ['row_id', 'data_block_id', 'target']]
oof = oof.set_index('row_id')
oof['target'] = 0

In [500]:
oof.head()

,data_block_id,target
row_id,,
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [461]:
FEATURES = [i for i in train_df.select_dtypes(include='number').columns if i != 'target']

In [462]:
FEATURES

['county',
 'is_business',
 'product_type',
 'is_consumption',
 'data_block_id',
 'row_id',
 'prediction_unit_id']

In [315]:
model = {}

In [456]:
# clean_df = train_df[~train_df['target'].isna()].reset_index(drop=True)
clean_df = train_df.fillna(value={'target':-999})

In [371]:
clean_df.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [458]:
clean_df.shape

(2018352, 9)

In [501]:
def predict_feature(model_dict, model_name, data, output_data, features):
    gkf = GroupKFold(n_splits=5)
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=data, groups=data.index)):
        print(f'Fold {i+1}')
        print('=======')
        train_x = data.loc[train_idx]
        train_y = data.loc[train_idx]
        valid = data.loc[valid_idx]
        valid_ids = valid.row_id.values

        reg = LinearRegression()
        reg.fit(train_x[features], train_y['target'])

        model_dict[f'{model_name}'] = reg 
        output_data.loc[valid_ids,['target']] = reg.predict(valid[features])
    return output_data

In [481]:
oof.shape

(2018352, 3)

In [502]:
oof = predict_feature(model, 'first', clean_df, oof, ['county'])

Fold 1
Fold 2
Fold 3
Fold 4
Fold 5


In [643]:
def mae(df_pred, df_train):
    return mean_absolute_error(oof['target'], clean_df['target'])

In [644]:
mae(oof, clean_df)

371.30336675567594

In [635]:
oof.shape[0]

2018352